In [1]:
# Librerías

import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra
import matplotlib.pyplot as plt # plotting
import seaborn as sns # plotting
import string # string manipulation
import re # regular expressions
import nltk # text manipulation
nltk.download('punkt')
nltk.download('stopwords')

from tqdm import trange # progress bar
from nltk import tokenize # text manipulation
from nltk.corpus import stopwords # text manipulation
from nltk.stem import WordNetLemmatizer # text manipulation
from nltk.probability import FreqDist # text manipulation
from collections import Counter # text manipulation
from sklearn.feature_extraction.text import CountVectorizer # text manipulation
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator # wordcloud generator
from IPython.display import display # image display
from PIL import Image

#hito 2
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn import tree

import joblib # guardar modelos

from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\felip\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\felip\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Como propuesta experimental, en este proyecto se hará uso de los métodos de clustering para la pregunta 2 y clasificación para la 1 y 3. Dado que clasificación es un método supervisado, adicionalmente se realizarán pruebas para obtener métricas de desempeño para escoger el mejor método de clasificación, mientras que, para el caso de clustering, al ser no supervisado, se deberán usar métodos para validar los clusters generados.

Para responder la primera pregunta, se ha realizado un preprocesamiento que consta de, en primer lugar, la conversión del rating a palabras, a modo de limitar las categorías de originalmente 5 a 3. En segundo lugar, se hace la limpieza del dataset de aquellas palabras poco útiles para nuestro propósito, por lo que se filtran números, stopwords y se actualiza todo el texto del dataset de reseñas a minúsculas. Dado que, aún con la conversión de ratings existen categorías que quedan subrepresentadas en comparación a otras, se hace un oversampling basado en el rating de las reseñas para obtener un balanceo de datos para que los modelos generados sean mas correctos.

In [2]:
# función para convertir el rating a palabras
def ratingTransform(rating):
    if rating <= 2:
        return "NEGATIVE"
    elif rating <= 4:
        return "NEUTRAL"
    else:
        return "POSITIVE"

# funcion para realizar la limpieza del texto en el dataset
def clean(review):
    review = review.lower()
    review = re.sub('[^a-z A-Z 0-9-]+', '', review)
    review = " ".join([word for word in review.split() if word not in stopwords.words('english')])
    return review

# funcion para hacer oversampling de los datos que se encuentran desbalanceados
def oversampling(X,y):
    ros = RandomOverSampler(random_state=42)
    X_resample, y_resample = ros.fit_resample(X, y)
    return (X_resample, y_resample)

Prosiguiendo con los N-gramas, se verá lo que ocurre con los 1-gramas. Primero, se procede con la vectorización, ya que se quiere convertir el texto a una forma que los algoritmos de aprendizaje que se usarán posteriormente puedan entender y procesar. En este caso, dado que en el preprocesamiento se estableció la necesidad de hacer oversampling a los datos, la vectorización será útil para luego poder hacer dicho oversampling. Finalmente, se realiza la división de los datos entre datos de prueba (con el 33% de los datos destinado para eso) y datos de entrenamiento. Adicionalmente, se usarán funciones extras otorgadas por la librería “joblib”, la cual será de utilidad para guardar los datos procesados, para no tener necesidad de ejecutar todo desde el principio, y asi conservar el entrenamiento. Esto es de especial importancia por lo que se mencionara a continuación.

Luego de almacenar la data útil para los algoritmos, se procede con el entrenamiento de los modelos. Para estos, se usarán principalmente los de clasificación, en particular Árboles de decisión, Naive Bayes, regresión logística y Support Vector Classifier. La elección de los modelos de clasificación esta guiada por el saber que tan viable es obtener una reseña positiva dados ciertos N-gramas. Ya que el entrenamiento es la parte mas pesada de este proceso, también se ha decidido el guardar la data mediante los comandos de joblib.

Cabe mencionar que en principio se realizarón comparaciones con respecto a los resultados obtenidos entre limitaciones a la vectorización eliminando todo aquel dato que tuviese baja representatividad y el dataset sin alteraciones. Los resultados para 1-gramas arrojaron las métricas que se muestran a continuación:


In [ ]:
# Agregar la data para 1-gramas

Como es posible observar, las métricas obtenidas al limitar la vectorización del dataset son comparablemente inferiores a las métricas obtenidas con el dataset sin restricciones. Por esta razón, en favor de buscar el resultado más adecuado, se ha decidido usar los datasets sin alteraciones, pese a los tiempos de ejecución más lentos.

Se procede a cargar los modelos con "joblib" y a extraer las principales métricas de desempeño para medir cual de los modelos ha sido el mas confiable con el entrenamiento dado. Estas métricas ayudarán en la comparación de modelos para decidir cual de ellos es mejor para poder responder la pregunta planteada. Tanto para 2-gramas como 3-gramas, se sigue el mismo procedimiento descrito anteriormente. Cabe resaltar la importancia de la librería joblib, ya que el procesamiento del entrenamiento de los modelos es una tarea conocida por ser pesada computacionalmente hablando. Queda en el anexo todo el código descrito en esta explicación.

Accuracy en test set: 0.753245910153207
              
              precision    recall  f1-score   support

    NEGATIVE       0.83      0.76      0.79      5135
     NEUTRAL       0.76      0.74      0.75      5135
    POSITIVE       0.69      0.76      0.73      5134

    accuracy                           0.75     15404
   
   macro avg       0.76      0.75      0.75     15404
weighted avg       0.76      0.75      0.75     15404



## Código Pregunta 1
### Preprocesamiento del dataset

In [3]:
data = pd.read_csv("threads.csv") # carga del dataset a usar como variable

In [4]:
# función para convertir el rating a palabras
def ratingTransform(rating):
    if rating <= 2:
        return "NEGATIVE"
    elif rating <= 4:
        return "NEUTRAL"
    else:
        return "POSITIVE"

# creacion del grafico de pie de las reviews segun rating
data["rating"] = data["rating"].apply(ratingTransform)

In [5]:
def clean(review):

    review = review.lower()
    review = re.sub('[^a-z A-Z 0-9-]+', '', review)
    review = " ".join([word for word in review.split() if word not in stopwords.words('english')])

    return review

data['review_description'] = data['review_description'].apply(clean)

In [6]:
Y = data['rating']

In [6]:
def oversampling(X,y):
    ros = RandomOverSampler(random_state=42)
    X_resample, y_resample = ros.fit_resample(X, y)
    return (X_resample, y_resample)


## 1-gram
**Dataset sin limitaciones**

In [8]:
vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(data["review_description"])

In [9]:
vectorized = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names_out())
X_1gram = vectorized

In [10]:
X_1gram, y1 = oversampling(X_1gram, Y)

In [11]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X_1gram, y1, test_size=0.33, random_state=37,stratify=y1)
joblib.dump(X1_train, 'modelos/X1_train.pkl')
joblib.dump(X1_test, 'modelos/X1_test.pkl')
joblib.dump(y1_train, 'modelos/y1_train.pkl')
joblib.dump(y1_test, 'modelos/y1_test.pkl')

['modelos/y1_test.pkl']

**Dataset con mindf = 0.0005**

In [7]:
vectorizer = CountVectorizer(min_df=0.0005)
vectors = vectorizer.fit_transform(data["review_description"])

In [8]:
vectorized = pd.DataFrame(vectors.toarray(), columns=vectorizer.get_feature_names_out())
X_11gram = vectorized

In [9]:
X_11gram, y11 = oversampling(X_11gram, Y)

In [10]:
X11_train, X11_test, y11_train, y11_test = train_test_split(X_11gram, y11, test_size=0.33, random_state=37,stratify=y11)
joblib.dump(X11_train, 'modelos/X11_train.pkl')
joblib.dump(X11_test, 'modelos/X11_test.pkl')
joblib.dump(y11_train, 'modelos/y11_train.pkl')
joblib.dump(y11_test, 'modelos/y11_test.pkl')

['modelos/y11_test.pkl']

### Entrenar Modelos
**Dataset sin limitaciones**

In [12]:
X1_train = joblib.load('modelos/X1_train.pkl')
y1_train = joblib.load('modelos/y1_train.pkl')

In [13]:
clf = DecisionTreeClassifier()
clf.fit(X1_train, y1_train) 
joblib.dump(clf, "modelos/1-gram_trained_DecisionTreeClassifier.joblib")

['modelos/1-gram_trained_DecisionTreeClassifier.joblib']

In [14]:
clf = MultinomialNB()
clf.fit(X1_train, y1_train)
joblib.dump(clf, "modelos/1-gram_trained_NV.joblib")

['modelos/1-gram_trained_NV.joblib']

In [15]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X1_train, y1_train)
joblib.dump(clf, "modelos/1-gram_logistic_regesion.joblib")

['modelos/1-gram_logistic_regesion.joblib']

In [16]:
clf = SVC()
clf.fit(X1_train, y1_train)
joblib.dump(clf, "modelos/1-gram_trained_svc.joblib")

['modelos/1-gram_trained_svc.joblib']

**Dataset con mindf = 0.0005**

In [11]:
X11_train = joblib.load('modelos/X11_train.pkl')
y11_train = joblib.load('modelos/y11_train.pkl')

In [12]:
clf = DecisionTreeClassifier()
clf.fit(X11_train, y11_train) 
joblib.dump(clf, "modelos/1-gram_trained_DecisionTreeClassifier_md.joblib")

['modelos/1-gram_trained_DecisionTreeClassifier_md.joblib']

In [13]:
clf = MultinomialNB()
clf.fit(X11_train, y11_train)
joblib.dump(clf, "modelos/1-gram_trained_NV_md.joblib")

['modelos/1-gram_trained_NV_md.joblib']

In [14]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X11_train, y11_train)
joblib.dump(clf, "modelos/1-gram_logistic_regesion_md.joblib")

['modelos/1-gram_logistic_regesion_md.joblib']

In [15]:
clf = SVC()
clf.fit(X11_train, y11_train)
joblib.dump(clf, "modelos/1-gram_trained_svc_md.joblib")

['modelos/1-gram_trained_svc_md.joblib']

### Cargar modelos para evaluar
**Dataset sin limitaciones**

In [34]:
X1_test = joblib.load('modelos/X1_test.pkl')
y1_test = joblib.load('modelos/y1_test.pkl')

In [35]:
clf = joblib.load("modelos/1-gram_trained_DecisionTreeClassifier.joblib")
y1_pred = clf.predict(X1_test)   ## Predecimos con nuevos datos (los de test X_test)
print("Accuracy en test set:", accuracy_score(y1_test, y1_pred))   ## Evaluamos la predicción comparando y_test con y1_pred
print(classification_report(y1_test, y1_pred))

Accuracy en test set: 0.753245910153207
              precision    recall  f1-score   support

    NEGATIVE       0.83      0.76      0.79      5135
     NEUTRAL       0.76      0.74      0.75      5135
    POSITIVE       0.69      0.76      0.73      5134

    accuracy                           0.75     15404
   macro avg       0.76      0.75      0.75     15404
weighted avg       0.76      0.75      0.75     15404



In [36]:
clf = joblib.load("modelos/1-gram_trained_NV.joblib")
y1_pred = clf.predict(X1_test)   ## Predecimos con nuevos datos (los de test X_test)
print("Accuracy en test set:", accuracy_score(y1_test, y1_pred))   ## Evaluamos la predicción comparando y_test con y1_pred
print(classification_report(y1_test, y1_pred))

Accuracy en test set: 0.6999480654375487
              precision    recall  f1-score   support

    NEGATIVE       0.80      0.71      0.75      5135
     NEUTRAL       0.68      0.56      0.62      5135
    POSITIVE       0.64      0.83      0.72      5134

    accuracy                           0.70     15404
   macro avg       0.71      0.70      0.70     15404
weighted avg       0.71      0.70      0.70     15404



In [37]:
clf = joblib.load("modelos/1-gram_logistic_regesion.joblib")
y1_pred = clf.predict(X1_test)   ## Predecimos con nuevos datos (los de test X_test)
print("Accuracy en test set:", accuracy_score(y1_test, y1_pred))   ## Evaluamos la predicción comparando y_test con y1_pred
print(classification_report(y1_test, y1_pred))

Accuracy en test set: 0.7331212672033238
              precision    recall  f1-score   support

    NEGATIVE       0.82      0.75      0.78      5135
     NEUTRAL       0.75      0.61      0.67      5135
    POSITIVE       0.66      0.84      0.74      5134

    accuracy                           0.73     15404
   macro avg       0.74      0.73      0.73     15404
weighted avg       0.74      0.73      0.73     15404



In [38]:
clf = joblib.load("modelos/1-gram_trained_svc.joblib")
y1_pred = clf.predict(X1_test)   ## Predecimos con nuevos datos (los de test X_test)
print("Accuracy en test set:", accuracy_score(y1_test, y1_pred))   ## Evaluamos la predicción comparando y_test con y1_pred
print(classification_report(y1_test, y1_pred))

Accuracy en test set: 0.729940275253181
              precision    recall  f1-score   support

    NEGATIVE       0.82      0.75      0.78      5135
     NEUTRAL       0.82      0.58      0.68      5135
    POSITIVE       0.63      0.86      0.73      5134

    accuracy                           0.73     15404
   macro avg       0.75      0.73      0.73     15404
weighted avg       0.75      0.73      0.73     15404



**Dataset con mindf = 0.0005**

In [20]:
X11_test = joblib.load('modelos/X11_test.pkl')
y11_test = joblib.load('modelos/y11_test.pkl')

In [21]:
clf = joblib.load("modelos/1-gram_trained_DecisionTreeClassifier_md.joblib")
y11_pred = clf.predict(X11_test)   ## Predecimos con nuevos datos (los de test X_test)
print("Accuracy en test set:", accuracy_score(y11_test, y11_pred))   ## Evaluamos la predicción comparando y_test con y1_pred
print(classification_report(y11_test, y11_pred))

Accuracy en test set: 0.7290963386133472
              precision    recall  f1-score   support

    NEGATIVE       0.81      0.73      0.77      5135
     NEUTRAL       0.74      0.69      0.72      5135
    POSITIVE       0.65      0.76      0.70      5134

    accuracy                           0.73     15404
   macro avg       0.74      0.73      0.73     15404
weighted avg       0.74      0.73      0.73     15404



In [22]:
clf = joblib.load("modelos/1-gram_trained_NV_md.joblib")
y11_pred = clf.predict(X11_test)   ## Predecimos con nuevos datos (los de test X_test)
print("Accuracy en test set:", accuracy_score(y11_test, y11_pred))   ## Evaluamos la predicción comparando y_test con y1_pred
print(classification_report(y11_test, y11_pred))

Accuracy en test set: 0.6627499350817969
              precision    recall  f1-score   support

    NEGATIVE       0.76      0.67      0.72      5135
     NEUTRAL       0.67      0.45      0.54      5135
    POSITIVE       0.60      0.86      0.70      5134

    accuracy                           0.66     15404
   macro avg       0.68      0.66      0.65     15404
weighted avg       0.68      0.66      0.65     15404



In [23]:
clf = joblib.load("modelos/1-gram_logistic_regesion_md.joblib")
y11_pred = clf.predict(X11_test)   ## Predecimos con nuevos datos (los de test X_test)
print("Accuracy en test set:", accuracy_score(y11_test, y11_pred))   ## Evaluamos la predicción comparando y_test con y1_pred
print(classification_report(y11_test, y11_pred))

Accuracy en test set: 0.6769021033497793
              precision    recall  f1-score   support

    NEGATIVE       0.77      0.70      0.73      5135
     NEUTRAL       0.69      0.49      0.57      5135
    POSITIVE       0.61      0.84      0.71      5134

    accuracy                           0.68     15404
   macro avg       0.69      0.68      0.67     15404
weighted avg       0.69      0.68      0.67     15404



In [24]:
clf = joblib.load("modelos/1-gram_trained_svc_md.joblib")
y11_pred = clf.predict(X11_test)   ## Predecimos con nuevos datos (los de test X_test)
print("Accuracy en test set:", accuracy_score(y11_test, y11_pred))   ## Evaluamos la predicción comparando y_test con y1_pred
print(classification_report(y11_test, y11_pred))

Accuracy en test set: 0.7174110620618022
              precision    recall  f1-score   support

    NEGATIVE       0.82      0.73      0.77      5135
     NEUTRAL       0.80      0.55      0.65      5135
    POSITIVE       0.61      0.87      0.72      5134

    accuracy                           0.72     15404
   macro avg       0.74      0.72      0.71     15404
weighted avg       0.74      0.72      0.71     15404



## 2-gram

In [8]:
cv = CountVectorizer(ngram_range=(2,2))
bigrams = cv.fit_transform(data['review_description'])

In [9]:
bigrams = pd.DataFrame(bigrams.toarray(), columns=cv.get_feature_names_out())
X_2gram = bigrams

In [10]:
X_2gram, y2 = oversampling(X_2gram, Y)

In [13]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X_2gram, y2, test_size=0.33, random_state=37,stratify=y2)
joblib.dump(X2_train, 'modelos/X2_train.pkl')
joblib.dump(X2_test, 'modelos/X2_test.pkl')
joblib.dump(y2_train, 'modelos/y2_train.pkl')
joblib.dump(y2_test, 'modelos/y2_test.pkl')

['modelos/y2_test.pkl']

### Entrenar Modelos

In [14]:
X2_train = joblib.load('modelos/X2_train.pkl')
y2_train = joblib.load('modelos/y2_train.pkl')

In [ ]:
clf = DecisionTreeClassifier()
clf.fit(X2_train, y2_train) 
joblib.dump(clf, "modelos/2-gram_trained_DecisionTreeClassifier.joblib")

['modelos/2-gram_trained_DecisionTreeClassifier.joblib']

In [ ]:
clf = MultinomialNB()
clf.fit(X2_train, y2_train)
joblib.dump(clf, "modelos/2-gram_trained_NV.joblib")

['modelos/2-gram_trained_NV.joblib']

In [ ]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X2_train, y2_train)
joblib.dump(clf, "modelos/2-gram_logistic_regesion.joblib")

['modelos/2-gram_logistic_regesion.joblib']

In [15]:
clf = SVC()
clf.fit(X2_train, y2_train)
joblib.dump(clf, "modelos/2-gram_trained_svc.joblib")

['modelos/2-gram_trained_svc.joblib']

### Cargar modelos para evaluar

In [ ]:
X2_test = joblib.load('modelos/X2_test.pkl')
y2_test = joblib.load('modelos/y2_test.pkl')

In [ ]:
clf = joblib.load("modelos/2-gram_trained_DecisionTreeClassifier.joblib")
y2_pred = clf.predict(X2_test)   ## Predecimos con nuevos datos (los de test X_test)
print("Accuracy en test set:", accuracy_score(y2_test, y2_pred))   ## Evaluamos la predicción comparando y2_test con y2_pred
print(classification_report(y2_test, y2_pred))

Accuracy en test set: 0.7294209296286679
              precision    recall  f1-score   support

    NEGATIVE       0.89      0.61      0.72      5135
     NEUTRAL       0.84      0.70      0.76      5135
    POSITIVE       0.59      0.88      0.71      5134

    accuracy                           0.73     15404
   macro avg       0.77      0.73      0.73     15404
weighted avg       0.77      0.73      0.73     15404



In [ ]:
clf = joblib.load("modelos/2-gram_trained_NV.joblib")
y2_pred = clf.predict(X2_test)   ## Predecimos con nuevos datos (los de test X_test)
print("Accuracy en test set:", accuracy_score(y2_test, y2_pred))   ## Evaluamos la predicción comparando y2_test con y2_pred
print(classification_report(y2_test, y2_pred))

Accuracy en test set: 0.7174759802648663
              precision    recall  f1-score   support

    NEGATIVE       0.88      0.61      0.72      5135
     NEUTRAL       0.80      0.66      0.72      5135
    POSITIVE       0.59      0.88      0.71      5134

    accuracy                           0.72     15404
   macro avg       0.76      0.72      0.72     15404
weighted avg       0.76      0.72      0.72     15404



In [ ]:
clf = joblib.load("modelos/2-gram_logistic_regesion.joblib")
y2_pred = clf.predict(X2_test)   ## Predecimos con nuevos datos (los de test X_test)
print("Accuracy en test set:", accuracy_score(y2_test, y2_pred))   ## Evaluamos la predicción comparando y2_test con y2_pred
print(classification_report(y2_test, y2_pred))

Accuracy en test set: 0.738314723448455
              precision    recall  f1-score   support

    NEGATIVE       0.90      0.63      0.74      5135
     NEUTRAL       0.84      0.68      0.75      5135
    POSITIVE       0.61      0.91      0.73      5134

    accuracy                           0.74     15404
   macro avg       0.78      0.74      0.74     15404
weighted avg       0.78      0.74      0.74     15404



In [ ]:
clf = joblib.load("modelos/2-gram_trained_svc.joblib")
y2_pred = clf.predict(X2_test)   ## Predecimos con nuevos datos (los de test X_test)
print("Accuracy en test set:", accuracy_score(y2_test, y2_pred))   ## Evaluamos la predicción comparando y2_test con y2_pred
print(classification_report(y2_test, y2_pred))

##  3-gram

In [18]:
cv1 = CountVectorizer(ngram_range=(3,3),min_df=0.00005)
trigrams = cv1.fit_transform(data['review_description'])

In [19]:
trigrams = pd.DataFrame(trigrams.toarray(), columns=cv1.get_feature_names_out())

In [20]:
X_3gram = trigrams

In [21]:
X_3gram, y3 = oversampling(X_3gram, Y)

In [22]:
X3_train, X3_test, y3_train, y3_test = train_test_split(X_3gram, y3, test_size=0.33, random_state=37,stratify=y3)
joblib.dump(X3_train, 'modelos/X3_train.pkl')
joblib.dump(X3_test, 'modelos/X3_test.pkl')
joblib.dump(y3_train, 'modelos/y3_train.pkl')
joblib.dump(y3_test, 'modelos/y3_test.pkl')

['modelos/y3_test.pkl']

In [23]:
X3_train = joblib.load('modelos/X3_train.pkl')
y3_train = joblib.load('modelos/y3_train.pkl')

In [26]:
clf = DecisionTreeClassifier()
clf.fit(X3_train, y3_train) 
joblib.dump(clf, "modelos/3-gram_trained_DecisionTreeClassifier.joblib")

['modelos/3-gram_trained_DecisionTreeClassifier.joblib']

In [27]:
clf = MultinomialNB()
clf.fit(X3_train, y3_train)
joblib.dump(clf, "modelos/3-gram_trained_NV.joblib")

['modelos/3-gram_trained_NV.joblib']

In [28]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X3_train, y3_train)
joblib.dump(clf, "modelos/3-gram_logistic_regesion.joblib")

['modelos/3-gram_logistic_regesion.joblib']

In [24]:
clf = SVC()
clf.fit(X3_train, y3_train)
joblib.dump(clf, "modelos/3-gram_trained_svc.joblib")

['modelos/3-gram_trained_svc.joblib']

### Cargar modelos para evaluar

In [ ]:
X3_test = joblib.load('modelos/X3_test.pkl')
y3_test = joblib.load('modelos/y3_test.pkl')

In [29]:
clf = joblib.load("modelos/3-gram_trained_DecisionTreeClassifier.joblib")
y3_pred = clf.predict(X3_test)   ## Predecimos con nuevos datos (los de test X_test)
print("Accuracy en test set:", accuracy_score(y3_test, y3_pred))   ## Evaluamos la predicción comparando y3_test con y3_pred
print(classification_report(y3_test, y3_pred))

Accuracy en test set: 0.5235003895092184
              precision    recall  f1-score   support

    NEGATIVE       0.85      0.26      0.40      5135
     NEUTRAL       0.81      0.37      0.50      5135
    POSITIVE       0.42      0.94      0.58      5134

    accuracy                           0.52     15404
   macro avg       0.69      0.52      0.50     15404
weighted avg       0.69      0.52      0.50     15404



In [30]:
clf = joblib.load("modelos/3-gram_trained_NV.joblib")
y3_pred = clf.predict(X3_test)   ## Predecimos con nuevos datos (los de test X_test)
print("Accuracy en test set:", accuracy_score(y3_test, y3_pred))   ## Evaluamos la predicción comparando y3_test con y3_pred
print(classification_report(y3_test, y3_pred))

Accuracy en test set: 0.508828875616723
              precision    recall  f1-score   support

    NEGATIVE       0.80      0.27      0.41      5135
     NEUTRAL       0.76      0.30      0.43      5135
    POSITIVE       0.42      0.95      0.58      5134

    accuracy                           0.51     15404
   macro avg       0.66      0.51      0.47     15404
weighted avg       0.66      0.51      0.47     15404



In [32]:
clf = joblib.load("modelos/3-gram_logistic_regesion.joblib")
y3_pred = clf.predict(X3_test)   ## Predecimos con nuevos datos (los de test X_test)
print("Accuracy en test set:", accuracy_score(y3_test, y3_pred))   ## Evaluamos la predicción comparando y3_test con y3_pred
print(classification_report(y3_test, y3_pred))

Accuracy en test set: 0.5246689171643729
              precision    recall  f1-score   support

    NEGATIVE       0.83      0.29      0.43      5135
     NEUTRAL       0.77      0.35      0.48      5135
    POSITIVE       0.43      0.94      0.59      5134

    accuracy                           0.52     15404
   macro avg       0.68      0.52      0.50     15404
weighted avg       0.68      0.52      0.50     15404



In [25]:
clf = joblib.load("modelos/3-gram_trained_svc.joblib")
y3_pred = clf.predict(X3_test)   ## Predecimos con nuevos datos (los de test X_test)
print("Accuracy en test set:", accuracy_score(y3_test, y3_pred))   ## Evaluamos la predicción comparando y3_test con y3_pred
print(classification_report(y3_test, y3_pred))

Accuracy en test set: 0.5309010646585303
              precision    recall  f1-score   support

    NEGATIVE       0.70      0.36      0.47      5135
     NEUTRAL       0.86      0.35      0.50      5135
    POSITIVE       0.42      0.89      0.57      5134

    accuracy                           0.53     15404
   macro avg       0.66      0.53      0.51     15404
weighted avg       0.66      0.53      0.51     15404

